<a href="https://colab.research.google.com/github/upnormalmekah/PRD-Maker-with-LLM/blob/main/TA_Pengolahan_citra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy==1.23.5 scikit-image==0.19.3 --quiet

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from google.colab.patches import cv2_imshow
from skimage.feature import greycomatrix, greycoprops
from skimage import filters

# Fungsi upload gambar
def upload_image():
    uploaded = files.upload()
    for filename in uploaded.keys():
        img = cv2.imread(filename)
        print(f"Gambar {filename} berhasil diupload.")
        return img
    return None

# ======== FITUR 1: OPERASI GEOMETRI ========
def operasi_geometri(img):
    rows, cols = img.shape[:2]
    print("1. Translasi\n2. Rotasi\n3. Scaling")
    pilih = input("Pilih operasi: ")

    if pilih == '1':
        dx = int(input("Translasi X: "))
        dy = int(input("Translasi Y: "))
        M = np.float32([[1, 0, dx], [0, 1, dy]])
        result = cv2.warpAffine(img, M, (cols, rows))

    elif pilih == '2':
        angle = float(input("Masukkan sudut rotasi (derajat): "))
        M = cv2.getRotationMatrix2D((cols/2, rows/2), angle, 1)
        result = cv2.warpAffine(img, M, (cols, rows))

    elif pilih == '3':
        fx = float(input("Skala X: "))
        fy = float(input("Skala Y: "))
        result = cv2.resize(img, None, fx=fx, fy=fy)

    else:
        print("Pilihan tidak valid.")
        return

    cv2_imshow(result)

# ======== FITUR 2: MORFOLOGI CITRA ========
def morfologi(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

    kernel = np.ones((5,5), np.uint8)
    print("1. Erosi\n2. Dilasi\n3. Opening\n4. Closing")
    pilih = input("Pilih operasi: ")

    if pilih == '1':
        result = cv2.erode(binary, kernel)
    elif pilih == '2':
        result = cv2.dilate(binary, kernel)
    elif pilih == '3':
        result = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
    elif pilih == '4':
        result = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
    else:
        print("Pilihan tidak valid.")
        return

    cv2_imshow(result)

# ======== FITUR 3: SEGMENTASI ========
def segmentasi(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    print("1. Sobel\n2. Prewitt\n3. Canny\n4. Deteksi Garis (Hough)")
    pilih = input("Pilih metode segmentasi: ")

    if pilih == '1':
        sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
        sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1)
        result = cv2.magnitude(sobelx, sobely)
    elif pilih == '2':
        kernelx = np.array([[1,0,-1],[1,0,-1],[1,0,-1]])
        kernely = np.array([[1,1,1],[0,0,0],[-1,-1,-1]])
        imgx = cv2.filter2D(gray, -1, kernelx)
        imgy = cv2.filter2D(gray, -1, kernely)
        result = cv2.addWeighted(imgx, 0.5, imgy, 0.5, 0)
    elif pilih == '3':
        result = cv2.Canny(gray, 100, 200)
    elif pilih == '4':
        edges = cv2.Canny(gray, 50, 150)
        lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=80, minLineLength=50, maxLineGap=10)
        result = img.copy()
        if lines is not None:
            for line in lines:
                x1,y1,x2,y2 = line[0]
                cv2.line(result, (x1,y1), (x2,y2), (0,255,0), 2)
    else:
        print("Pilihan tidak valid.")
        return

    cv2_imshow(result)

# ======== FITUR 4: GLCM ========
def glcm(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray16 = (gray / 16).astype(np.uint8)
    glcm = greycomatrix(gray16, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=16, symmetric=True, normed=True)

    fitur = {
        'Contrast': greycoprops(glcm, 'contrast').mean(),
        'Dissimilarity': greycoprops(glcm, 'dissimilarity').mean(),
        'Homogeneity': greycoprops(glcm, 'homogeneity').mean(),
        'Energy': greycoprops(glcm, 'energy').mean(),
        'Correlation': greycoprops(glcm, 'correlation').mean(),
        'ASM': greycoprops(glcm, 'ASM').mean()
    }

    print("\nFitur GLCM:")
    for k, v in fitur.items():
        print(f"{k}: {v:.4f}")

# ======== MAIN PROGRAM ========
def main():
    print("=== APLIKASI PENGOLAHAN CITRA DIGITAL ===")
    img = upload_image()
    if img is None:
        print("Upload gambar gagal.")
        return

    while True:
        print("\nMenu:")
        print("1. Operasi Geometri")
        print("2. Morfologi Citra")
        print("3. Segmentasi Citra")
        print("4. Analisis GLCM")
        print("5. Keluar")
        pilihan = input("Pilih fitur (1-5): ")

        if pilihan == '1':
            operasi_geometri(img)
        elif pilihan == '2':
            morfologi(img)
        elif pilihan == '3':
            segmentasi(img)
        elif pilihan == '4':
            glcm(img)
        elif pilihan == '5':
            print("Keluar dari aplikasi.")
            break
        else:
            print("Pilihan tidak valid!")

main()


=== APLIKASI PENGOLAHAN CITRA DIGITAL ===


Saving gundam.jpg to gundam (1).jpg
Gambar gundam (1).jpg berhasil diupload.

Menu:
1. Operasi Geometri
2. Morfologi Citra
3. Segmentasi Citra
4. Analisis GLCM
5. Keluar
Pilih fitur (1-5): 4


/usr/local/lib/python3.11/dist-packages/skimage/feature/__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  removed_version='1.0')
/usr/local/lib/python3.11/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.11/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.11/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.11/dist-packages/skimag


Fitur GLCM:
Contrast: 0.9583
Dissimilarity: 0.4067
Homogeneity: 0.8420
Energy: 0.2589
Correlation: 0.9489
ASM: 0.0672

Menu:
1. Operasi Geometri
2. Morfologi Citra
3. Segmentasi Citra
4. Analisis GLCM
5. Keluar
Pilih fitur (1-5): 5
Keluar dari aplikasi.
